## package prepare

In [184]:
import json
import pandas as pd
import demjson3
import subprocess
import re

## data prepare - writing

In [185]:
# with open("test_cases_jsonc.json", "r", encoding="utf-8") as f:
#     df_test_cases_jsonc_load = json.load(f)
# # Create DataFrame
# df_test_cases_jsonc = pd.DataFrame(df_test_cases_jsonc_load)

# modifying_config=pd.read_csv('/Users/shiyi.he/Desktop/PARSER/modifying_config.csv')
# modifying_tabular=pd.read_csv('/Users/shiyi.he/Desktop/PARSER/modifying_tabular.csv')


writing_config=pd.read_csv('/Users/shiyi.he/Desktop/PARSER/writing_norm_config.csv')
writing_tabular=pd.read_csv('/Users/shiyi.he/Desktop/PARSER/writing_norm_tabular.csv')

writing_config_json=writing_config[['participantId','format','code']][writing_config['format']=='json']
writing_tabular_json=writing_tabular[['participantId','format','code']][writing_tabular['format']=='json']
writing_config_jsonc=writing_config[['participantId','format','code']][writing_config['format']=='jsonc']
writing_tabular_jsonc=writing_tabular[['participantId','format','code']][writing_tabular['format']=='jsonc']
writing_config_json5=writing_config[['participantId','format','code']][writing_config['format']=='json5']
writing_tabular_json5=writing_tabular[['participantId','format','code']][writing_tabular['format']=='json5']
writing_config_hjson=writing_config[['participantId','format','code']][writing_config['format']=='hjson']
writing_tabular_hjson=writing_tabular[['participantId','format','code']][writing_tabular['format']=='hjson']
writing_config_xml=writing_config[['participantId','format','code']][writing_config['format']=='xml']
writing_tabular_xml=writing_tabular[['participantId','format','code']][writing_tabular['format']=='xml']
writing_config_yaml=writing_config[['participantId','format','code']][writing_config['format']=='yaml']
writing_tabular_yaml=writing_tabular[['participantId','format','code']][writing_tabular['format']=='yaml']
writing_config_toml=writing_config[['participantId','format','code']][writing_config['format']=='toml']
writing_tabular_toml=writing_tabular[['participantId','format','code']][writing_tabular['format']=='toml']

## global functions

In [ ]:
def remove_comments_from_string(s: str) -> str:
    """
    Remove both single-line (//...) and multi-line (/*...*/) comments from a string.
    """
    # Remove /* ... */ comments (including newlines)
    s = re.sub(r'/\*[\s\S]*?\*/', '', s)
    # Remove // comments, up to the end of the line
    s = re.sub(r'//.*?$', '', s, flags=re.MULTILINE)
    return s

def preprocess_remove_comments(
    df: pd.DataFrame,
    code_col: str = "code",
    out_col: str = "code_no_comments"
) -> pd.DataFrame:
    """
    Given a DataFrame with a column of code strings, produce a copy with an
    extra column where all //... and /*...*/ comments have been stripped.
    """
    df2 = df.copy()
    df2[out_col] = df2[code_col].apply(remove_comments_from_string)
    return df2

# —— clean comments example ——  
# 假设你已有 df_result_wt 包含 'code' 列
# df_clean = preprocess_remove_comments(writing_config_jsonc, code_col="code", out_col="clean_code")
# df_clean = (
#     df_clean
#     .drop(columns=["code"])               # 删除旧的 code 列
#     .rename(columns={"clean_code": "code"})  # 将 clean_code 重命名为 code
# )

In [ ]:
def print_code_and_error(
    df: pd.DataFrame,
    participant_col: str,
    code_col: str,
    error_col: str,
    pid: str
) -> None:
    """
    打印指定 participantId 的完整 code 和对应的 error message。

    参数
    ----
    df : pandas.DataFrame
        包含 participant_col, code_col, error_col 列的数据表。
    participant_col : str
        标识列名，如 "participantId"。
    code_col : str
        存放原始代码文本的列名，如 "code"。
    error_col : str
        存放错误消息的列名，如 "errorMessage"。
    pid : str
        要查询的 participantId 值。
    """
    # 过滤到该 participantId 的行
    sub = df[df[participant_col] == pid]
    if sub.empty:
        print(f"No entry found for participantId = {pid}")
        return

    # 取第一条（假设唯一）
    row = sub.iloc[0]
    code = row[code_col]
    err  = row[error_col]

    print(f"--- participantId: {pid} ---\n")
    print(">>> Full code:")
    print(code)
    print("\n>>> Error message:")
    print(err or "(no error)")


# 方法一：直接打印
# mask = df_result_wt["participantId"] == pid
# if mask.any():
#     print(df_result_wt.loc[mask, "errorMessage"].iloc[0])
# else:
#     print(f"No entry for participantId {pid}")

# 方法二：封装成函数
def print_error_message(df, pid):
    sub = df[df["participantId"] == pid]
    if sub.empty:
        print(f"No entry for participantId {pid}")
    else:
        print(f"Errors for {pid}:")
        print(sub["errorMessage"].iloc[0])


In [ ]:
import pandas as pd

def process_and_save_parsing_results(
    df_input,
    format_name,
    validate_func,
    summarize_strict_func,
    output_dir="/Users/shiyi.he/Desktop/PARSER/result_writing"
):
    # 1️⃣ 严格解析
    df_strict = validate_func(df_input)
    df_strict_summary = summarize_strict_func(df_strict)
    strict_merged = pd.merge(df_strict, df_strict_summary, on='participantId', how='left')
    strict_path = f"{output_dir}/strict_parse_{format_name}.csv"
    strict_merged.to_csv(strict_path, index=False)

    # 2️⃣ 宽松解析
    df_loose = parse_loose_json(df_input, code_col="code")
    df_loose_result = df_loose[["participantId", "valid", "parsed", "errorMessage"]]
    df_loose_summary = summarize_loose_errors(df_loose_result)
    loose_merged = pd.merge(df_loose_result, df_loose_summary, on='participantId', how='left')
    loose_path = f"{output_dir}/loose_parse_{format_name}.csv"
    loose_merged.to_csv(loose_path, index=False)

    # 3️⃣ 拼接 strict + loose validity
    df_strict = df_strict.rename(columns={'valid': 'valid_strict'})
    df_loose_result = df_loose_result.rename(columns={'valid': 'valid_loose'})
    df1_subset = df_strict[['participantId', 'format', 'valid_strict']]
    df2_subset = df_loose_result[['participantId', 'valid_loose']]
    final_merged = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
    final_path = f"{output_dir}/parser_rwsult_{format_name}.csv"
    final_merged.to_csv(final_path, index=False)

    return final_merged


## jsonc syntax check

In [145]:
### jsonc

def validate_jsonc(
    df: pd.DataFrame,
    script_path: str = "/Users/shiyi.he/Desktop/PARSER/validate_jsonc.js",
    participant_col: str = "participantId",
    format_col: str = "format",
    code_col: str = "code"
) -> pd.DataFrame:
    # 1. 准备输入
    records = df[[participant_col, format_col, code_col]].to_dict(orient="records")
    input_str = json.dumps(records)

    # 2. 调用脚本（使用完整路径）
    proc = subprocess.run(
        ["/Users/shiyi.he/bin/node", script_path],
        input=input_str.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    if proc.returncode != 0:
        raise RuntimeError(f"Node.js 校验脚本出错：{proc.stderr.decode()}")

    # 3. 解析输出并合并
    validated = json.loads(proc.stdout.decode("utf-8"))
    df_valid = pd.DataFrame(validated)

    # 保证列名一致
    df_valid.rename(columns={participant_col: participant_col}, inplace=True)

    # 4. 合并
    df_merged = df.merge(
        df_valid[[participant_col, "valid", "errorMessage"]],
        on=participant_col,
        how="left"
    )
    return df_merged


def summarize_jsonc_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    对每个 participant 统计他们各自犯了哪些错误，以及每种错误出现了多少次。
    
    返回一个宽格式的 DataFrame，行索引是 participantId，
    列是不同的 errorType，值是出现次数。
    """
    # 1. 构造长格式列表
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        errs = row[error_col]
        if not errs or pd.isna(errs):
            continue
        # 拆分；再去掉“ at offset...” 的后缀
        for err in errs.split("; "):
            err_type = err.split(" at ")[0].strip()
            records.append({"participantId": pid, "errorType": err_type})
    
    if not records:
        return pd.DataFrame()  # 没记录就返回空表
    
    long_df = pd.DataFrame(records)
    
    # 2. 用 groupby + size 统计
    summary = (
        long_df
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    
    # 3. 如果你想要宽格式（pivot table）：
    pivot = summary.pivot(
        index=participant_col,
        columns="errorType",
        values="count"
    ).fillna(0).astype(int)
    
    return pivot


def parse_loose_json(df, code_col="code"):
    """
    对 df[code_col] 中的字符串做超级宽松 JSONC/JSON5/HJSON 解析。
    返回：
      - parsed：解析结果或 None
      - parse_error：错误信息或 None
      - parsed_valid：True/False，表示是否成功解析
    """
    parsed, errors, valid = [], [], []
    for s in df[code_col]:
        try:
            parsed.append(demjson3.decode(s))
            errors.append(None)
            valid.append(True)
        except Exception as e:
            parsed.append(None)
            errors.append(str(e))
            valid.append(False)

    df2 = df.copy()
    df2["parsed"] = parsed
    df2["errorMessage"] = errors
    df2["valid"] = valid
    return df2


def summarize_loose_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    对 df 中的 parse_error 列做宽松解析错误汇总。
    返回一个 pivot 表：行是 participantId，列是每种 errorType，值是次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        err = row[error_col]
        # 只统计真正出错的情况
        if pd.notna(err) and err:
            # demjson3.decode 出错时，err 就是一段消息，我们直接把它当作 errorType
            records.append({
                participant_col: pid,
                "errorType": err
            })
    if not records:
        return pd.DataFrame()  # 如果没有任何错误，返回空表

    long = pd.DataFrame(records)
    # 统计每个 participantId + errorType 的出现次数
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    # Pivot 成宽表
    pivot = summary.pivot(
        index=participant_col,
        columns="errorType",
        values="count"
    ).fillna(0).astype(int)
    return pivot

In [137]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_jsonc=validate_jsonc(writing_tabular_jsonc)
sum_jsonc=summarize_jsonc_errors(df_result_jsonc)
strict_parse_jsonc = pd.merge(df_result_jsonc, sum_jsonc, on='participantId', how='left')
strict_parse_jsonc.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_jsonc.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_jsonc = parse_loose_json(writing_tabular_jsonc, code_col="code")
df_loose__result_jsonc=df_loose_jsonc[["participantId","valid","parsed","errorMessage"]]
summary_loose_jsonc = summarize_loose_errors(df_loose__result_jsonc)

loose_parse_jsonc = pd.merge(df_loose__result_jsonc, summary_loose_jsonc, on='participantId', how='left')
loose_parse_jsonc.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_jsonc.csv', index=False)

## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_jsonc = df_result_jsonc.rename(columns={'valid': 'valid_strict'})
df_loose__result_jsonc = df_loose__result_jsonc.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_jsonc[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_jsonc[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_jsonc = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_jsonc.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_rwsult_jsonc.csv', index=False)
parser_rwsult_jsonc

,participantId,format,valid_strict,valid_loose
0,67f5bb69760b211cdfe8021c,jsonc,False,False
1,678bad003d15e6aba1ee5a5d,jsonc,True,True
2,67f0c5d37f91ceae5cb3b318,jsonc,False,False
3,67aee0dfad3492b61a9ba0eb,jsonc,False,False
4,6734346ca8e90c2da6fb4509,jsonc,True,True
5,67ab214c2cde1be7c45535ac,jsonc,False,False
6,607728dc0b4f8590b94ae57c,jsonc,False,False
7,5dafea4de40355001651fa2f,jsonc,False,False
8,677d185800ab83d1b0185d82,jsonc,False,False
9,6784ff2095dd9b97e969b01f,jsonc,False,False


## hjson

In [138]:
import json
import subprocess
import pandas as pd
import hjson
import demjson3

# ========== 1. 严格 HJSON 解析（HJSON 专用） ==========
def validate_hjson(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    对每条 HJSON 代码做严格校验，忽略“Found whitespace in your key name”错误，
    新增两列：
      - valid (bool)
      - errorMessage (str or None)
    """
    valid_list, error_list = [], []
    for s in df[code_col]:
        try:
            hjson.loads(s)
            valid_list.append(True)
            error_list.append(None)
        except Exception as e:
            msg = str(e)
            # 忽略键名中空格的错误
            if "Found whitespace in your key name" in msg:
                valid_list.append(True)
                error_list.append(None)
            else:
                valid_list.append(False)
                error_list.append(msg)
    df2 = df.copy()
    df2["valid"] = valid_list
    df2["errorMessage"] = error_list
    return df2

def summarize_hjson_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将 strict_error 拆成长表并 pivot 成宽表：
    行是 participantId，列是 errorType，值是次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        err = row[error_col]
        if pd.notna(err) and err:
            # 只保留冒号前面的主要报错描述
            et = err.split(":", 1)[0].strip()
            records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    pivot = (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )
    return pivot

In [139]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_hjson=validate_hjson(writing_tabular_hjson)
sum_hjson=summarize_hjson_errors(df_result_hjson)
strict_parse_hjson = pd.merge(df_result_hjson, sum_hjson, on='participantId', how='left')
strict_parse_hjson.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_hjson.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_hjson = parse_loose_json(writing_tabular_hjson, code_col="code")
df_loose__result_hjson=df_loose_hjson[["participantId","valid","parsed","errorMessage"]]
summary_loose_hjson = summarize_loose_errors(df_loose__result_hjson)

loose_parse_hjson = pd.merge(df_loose__result_hjson, summary_loose_hjson, on='participantId', how='left')
loose_parse_hjson.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_hjson.csv', index=False)

## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_hjson = df_result_hjson.rename(columns={'valid': 'valid_strict'})
df_loose__result_hjson = df_loose__result_hjson.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_hjson[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_hjson[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_hjson = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_hjson.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_hjson.csv', index=False)
parser_rwsult_hjson

,participantId,format,valid_strict,valid_loose
0,6697c8b6d9d7baa7edd8c4d3,hjson,True,False
1,678f0bb28ec3307e4f1afb78,hjson,True,False
2,60a0b744298c64d46b63e893,hjson,False,False
3,66cdfebcb908ea2717d06b6c,hjson,False,False
4,56cb8858edf8da000b6df354,hjson,False,False
5,667971a555af1f83d3f29ae7,hjson,True,False
6,5c6414540821d30001046198,hjson,False,False
7,667c18ace97ab6869cb1c50e,hjson,False,False
8,67a754a5c8614c52a939d87b,hjson,True,False
9,65cf6d8a589a67afcab54e6f,hjson,True,False


## json5

In [140]:
import pandas as pd
import pyjson5
import demjson3

# ——— 1. 严格 JSON5 解析 ———
def validate_json5(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    用 pyjson5 对每条 code 做严格 JSON5 解析，
    返回新增两列：
      - valid (bool)
      - errorMessage (str or None)
    """
    valid_list, error_list = [], []
    for s in df[code_col]:
        try:
            pyjson5.decode(s)
            valid_list.append(True)
            error_list.append(None)
        except Exception as e:
            valid_list.append(False)
            error_list.append(str(e))
    df2 = df.copy()
    df2["valid"] = valid_list
    df2["errorMessage"] = error_list
    return df2

def summarize_json5_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将严格 JSON5 解析的 errorMessage 拆分并 pivot 成宽表：
    行是 participantId，列是 errorType，值为出现次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        em = row[error_col]
        if pd.notna(em) and em:
            # 取第一行错误摘要，再按冒号截断保留主要信息
            et = em.split("\n", 1)[0].split(":", 1)[0].strip()
            records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    pivot = (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )
    return pivot



In [141]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_json5=validate_json5(writing_tabular_json5)
sum_json5=summarize_json5_errors(df_result_json5)
strict_parse_json5 = pd.merge(df_result_json5, sum_json5, on='participantId', how='left')
strict_parse_json5.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_json5.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_json5 = parse_loose_json(writing_tabular_json5, code_col="code")
df_loose__result_json5=df_loose_json5[["participantId","valid","parsed","errorMessage"]]
summary_loose_json5 = summarize_loose_errors(df_loose__result_json5)

loose_parse_json5 = pd.merge(df_loose__result_json5, summary_loose_json5, on='participantId', how='left')
loose_parse_json5.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_json5.csv', index=False)

## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_json5 = df_result_json5.rename(columns={'valid': 'valid_strict'})
df_loose__result_json5 = df_loose__result_json5.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_json5[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_json5[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_json5 = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_json5.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_json5.csv', index=False)
parser_rwsult_json5

,participantId,format,valid_strict,valid_loose
0,64457bc906c125cebd4bf66b,json5,False,False
1,673e43cc760355e9e4f042a3,json5,True,True
2,6788c1c724c24dc56b08790a,json5,False,False
3,67c246023b44ca64d40ae45a,json5,False,False
4,672d98919cc177d28c204e76,json5,False,False
5,67f1a4f8dd44e7cd5adffa29,json5,False,False
6,672ca6c0324ac26cdc0dd298,json5,False,False
7,5b68c9eb87af310001584803,json5,False,False
8,67e17bac09359e09118892b4,json5,False,False
9,6606b31155457e4f162ecf5a,json5,True,True


## json

In [142]:
import json
import pandas as pd
import demjson3

# ——— 1. 严格 JSON 解析 ———
def validate_json(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    对每条 code 做严格 JSON 解析（json.loads），
    返回新增两列：
      - valid (bool)
      - errorMessage (str or None)
    """
    valid_list, error_list = [], []
    for s in df[code_col]:
        try:
            json.loads(s)
            valid_list.append(True)
            error_list.append(None)
        except Exception as e:
            valid_list.append(False)
            error_list.append(str(e))
    df2 = df.copy()
    df2["valid"] = valid_list
    df2["errorMessage"] = error_list
    return df2

def summarize_json_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将严格 JSON 解析的 errorMessage 拆分并 pivot 成宽表：
    行是 participantId，列是 errorType，值为出现次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        em = row[error_col]
        if pd.notna(em) and em:
            # 取第一行错误摘要，再按冒号截断保留主要信息
            et = em.split("\n", 1)[0].split(":", 1)[0].strip()
            records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    pivot = (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )
    return pivot



In [146]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_json=validate_json(writing_tabular_json)
sum_json=summarize_json_errors(df_result_json)
strict_parse_json = pd.merge(df_result_json, sum_json, on='participantId', how='left')
strict_parse_json.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_json.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_json = parse_loose_json(writing_tabular_json, code_col="code")
df_loose__result_json=df_loose_json[["participantId","valid","parsed","errorMessage"]]
summary_loose_json = summarize_loose_errors(df_loose__result_json)

loose_parse_json = pd.merge(df_loose__result_json, summary_loose_json, on='participantId', how='left')
loose_parse_json.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_json.csv', index=False)

## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_json = df_result_json.rename(columns={'valid': 'valid_strict'})
df_loose__result_json = df_loose__result_json.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_json[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_json[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_json = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_json.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_json.csv', index=False)
parser_rwsult_json

,participantId,format,valid_strict,valid_loose
0,66475d6f7b27e4443ef0d031,json,False,False
1,67e571bef6af7ecd9c29ad72,json,True,True
2,60f1f1d4ac991a259b83f603,json,False,False
3,673c13c9d37c8a73bc80b8ce,json,False,False
4,667ac09492ea0caadf488520,json,False,False
5,67debe02a893eea51e8d02f2,json,False,False
6,615200d54efc832849db2259,json,False,False
7,66b0d065329001a04e5ae671,json,False,False
8,5e3b5fa9255e7a37bc841135,json,False,False
9,5d30dfa68b1523000134878f,json,False,False


In [147]:
df_loose__result_json

,participantId,valid_loose,parsed,errorMessage
5,66475d6f7b27e4443ef0d031,False,None,"Missing value for object property, expected "":"""
8,67e571bef6af7ecd9c29ad72,True,"{'patients': [{'name': 'John', 'tests': [{'id': 1, 'results': 'Normal'}], 'treatment': 'Regular Monitoring'}, {'name': 'Michael', 'tests': [{'id': 1, 'results': 'Elevated Cholesterol'}, {'id': 1, 'results': 'Normal'}]}]}",None
9,60f1f1d4ac991a259b83f603,False,None,"('Unknown identifier', 'Note')"
12,673c13c9d37c8a73bc80b8ce,False,None,Values must be separated by a comma
34,667ac09492ea0caadf488520,False,None,"('Unknown identifier', 'Name')"
37,67debe02a893eea51e8d02f2,False,None,"Missing value for object property, expected "":"""
44,615200d54efc832849db2259,False,None,"('Line terminator characters must be escaped inside string literals', 'U+000D')"
52,66b0d065329001a04e5ae671,False,None,Values must be separated by a comma
54,5e3b5fa9255e7a37bc841135,False,None,Values must be separated by a comma
56,5d30dfa68b1523000134878f,False,None,Values must be separated by a comma


## xml

In [149]:
import pandas as pd
from lxml import etree

# ——— 1. 严格 XML 解析 ———
_amp_pattern = re.compile(r'&(?!#?\w+;)')

def escape_amp(s: str) -> str:
    return _amp_pattern.sub("&amp;", s)

def validate_xml(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    对每条 XML 文本做严格解析（recover=False），
    返回新增两列：
      - valid (bool)
      - errorMessage (str or None)
    """
    valid_list = []
    error_list = []
    
    for xml_str in df[code_col]:
        # 1) 先做最基本的裸 & 转义
        cleaned = escape_amp(xml_str)
        
        # 2) 每次都新建一个 parser，避免保留上一次的状态
        parser = etree.XMLParser(recover=False, resolve_entities=False)
        try:
            etree.fromstring(cleaned.encode("utf-8"), parser=parser)
            valid_list.append(True)
            error_list.append(None)
        except Exception as e:
            valid_list.append(False)
            error_list.append(str(e))
    
    df2 = df.copy()
    df2["valid"] = valid_list
    df2["errorMessage"] = error_list
    return df2

def summarize_xml_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将严格 XML 解析的 errorMessage 拆分并 pivot 成宽表：
    行是 participantId，列是 errorType，值为出现次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        em = row[error_col]
        if pd.notna(em) and em:
            # 按行或分号拆分错误，然后取主要信息
            for part in em.splitlines():
                et = part.strip()
                if et:
                    records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    pivot = (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )
    return pivot

def parse_loose_xml(
    df, participant_col="participantId", code_col="code"
):
    valids, errors = [], []
    for xml_str in df[code_col]:
        parser = etree.XMLParser(recover=True)
        try:
            tree = etree.fromstring(xml_str.encode("utf-8"), parser=parser)
            if tree is None:
                raise ValueError("Could not recover any root element")
            valids.append(True)
            errors.append(None)
        except Exception as e:
            valids.append(False)
            errors.append(str(e))
    df2 = df.copy()
    df2["valid"] = valids
    df2["errorMessage"]  = errors
    return df2
    
def summarize_loose_xml_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "parse_error"
) -> pd.DataFrame:
    """
    将宽松解析的 parse_error 拆分并 pivot 成宽表：
    行是 participantId，列是 errorType，值为出现次数。
    """
    records = []
    for _, row in df.iterrows():
        pid = row[participant_col]
        pe = row[error_col]
        if pd.notna(pe) and pe:
            for part in str(pe).split("; "):
                et = part.strip()
                records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    pivot = (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )
    return pivot



In [194]:
df_result_xml=validate_xml(writing_config_xml)
df_result_xml

,participantId,format,code,valid,errorMessage
4,57ca4b09993b08000192297b,xml,"<version>4.4.7</version>\r\n<repository>\r\n <author>\r\n <middle_name=""""/>\r\n <IsMember>true</IsMember>\r\n </author>\r\n <type>git</type>\r\n</repository>\r\n<engines>\r\n <pnpm> >=8 </pnpm>\r\n</engines>\r\n<keywords>canvas</keywords>\r\n<keywords>responsive</keywords>\r\n<keywords>charts</keywords>",False,"Extra content at the end of the document, line 2, column 1 (<string>, line 2)"
6,6693ce87e76d32ccf07fbcd8,xml,"<?xml version = ""4.47"" encoding = ""UTF-8""?>\r\n<repository>\r\n <author>\r\n <middle_name></middle>\r\n <IsMember>true</IsMember>\r\n </author>\r\n <type>git</type>\r\n</repository>\r\n<engines>\r\n <pnpon> >=8 </pnpon>\r\n</engines>\r\n<keywords>\r\n <canvas></canvas>\r\n <responsive></responsive>\r\n <charts></charts>\r\n</keywords>",False,"Unsupported version '4.47', line 1, column 23 (<string>, line 1)"
26,60f70af563ef757f65e750e0,xml,</version> \r\n <4.4.7>\r\n</repository>\r\n <author>middle name</author>\r\n <IsMember>true</IsMember>\r\n</engines>\r\n <pnpm>>=8</pnpm>\r\n</keywords>\r\n <canvas> \r\n </responsive>\r\n </charts>\r\n,False,"StartTag: invalid element name, line 1, column 2 (<string>, line 1)"
44,67c045fa5074c5500ca40e76,xml,<root>\r\n<version>4.4.7</version>\r\n<respository>\r\n<authot>\r\n<middle_name></LsMember</middle_name>\r\n<isMember>true></isMember>\r\n</author>\r\n<type>git</type>\r\n</repository>\r\n<engines>\r\n <npm>>=6</npm>\r\n </engines>\r\n <keywords>\r\n <keyword>canvas</keyword>\r\n <keyword>responsive</keyword>\r\n <keyword>charts</keyword>\r\n </keywords>\r\n \r\n,False,"expected '>', line 5, column 24 (<string>, line 5)"
45,67d2eaf9d5996dc6be1c7072,xml,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<root>\r\n <version>4.4.7</version>\r\n <type>pnpm</type>\r\n </repository>\r\n <keywords>canvas</keywords>\r\n <keywords>responsive</keywords>\r\n <keywords>charts</keywords>",False,"Opening and ending tag mismatch: root line 2 and repository, line 5, column 18 (<string>, line 5)"
62,651b29ff3a6cadf518640abe,xml,"<？xml version=""4.4.7"" encoding></？xml>\r\n<repository></repository>author</middle_name>\r\n<repository></repository>type<git>\r\n author<middle_name></middle>\r\n <IsMember></IsMember=true>\r\n<engines></engines>pnpm>=8\r\n<keywords></keywords>\r\n <canvas></canvas>\r\n <responsive></responsive>\r\n <charts></charts>\r\n",False,"Specification mandates value for attribute encoding, line 1, column 31 (<string>, line 1)"
69,67aca6831f2b0b79416bdb6c,xml,"<?xml version=""1.0""?> <root></root>\r\n<root>\r\n<version>2.4.7/versio>\r\n<repository><engineer>\r\n><keywords><propan/></keywords></\r\nrespository\r\n</engineer> <root></root>",False,"Extra content at the end of the document, line 2, column 1 (<string>, line 2)"
79,677f8968bbb08bc37c8d2a4b,xml,"<data>\r\n <element name=""keyword""></element>\r\n <git=3.0>\r\n <engines=3.0>\r\n </PNPM=8.0>\r\n <git=>=8</git>\r\n <element name=""metadata""> \r\n </element>\r\n",False,"error parsing attribute name, line 3, column 11 (<string>, line 3)"
80,672f51ea346fba23ddd5a793,xml,<version>4.4.7</version>\r\n <repository>\r\n <auhor>\r\n <middle_name/>\r\n <isMember>true</isMember>\r\n </author>\r\n <type>git</type>\r\n </repository>\r\n <engines>\r\n <npm>7</npm>\r\n <yarn>1</yarn>\r\n </engines>\r\n <keywords>\r\n <keyword>canvas</keyword>\r\n <keyword>responsive</keyword>\r\n <keyword>charts<keyword></keyword>,False,"Extra content at the end of the document, line 2, column 5 (<string>, line 2)"
90,62d7fef5538b81f205ba4656,xml,"<?xml version=""4.4.7"" encoding=""UTF-8"">\r\n<repository>\r\n<author>\r\n<middle_name></middle_name>\r\n<isMember>True</isMember>\r\n</author>\r\n<type>Git</type>\r\n<engines>\r\n<pnpm>>=8</pnpm>\r\n<keywords>\r\n<canvas></canvas>\r\n<responsive></responsive>\r\n<charts></charts>\r\n</keywords>\r\n</engines>\r\n</repository>",False,"String not closed expecting "" or ', line 1, column 19 (<string>, line 1)"


In [197]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_xml=validate_xml(writing_tabular_xml)
sum_xml = summarize_xml_errors(df_result_xml)
sum_xml = sum_xml.reset_index()
# 3. （可选）把第一列改个名字，确保它叫 participantId
sum_xml = sum_xml.rename(columns={sum_xml.columns[0]: 'participantId'})
print(sum_xml.columns.tolist())

strict_parse_xml = pd.merge(df_result_xml, sum_xml, on='participantId', how='left')
strict_parse_xml.to_csv('/Users/shiyi.he/Desktop/PARSER/strict_parse_xml.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_xml = parse_loose_xml(writing_tabular_xml, code_col="code")
df_loose__result_xml=df_loose_xml[["participantId","valid","code","errorMessage"]]
summary_loose_xml = summarize_loose_errors(df_loose__result_xml)

# loose_parse_xml = pd.merge(df_loose__result_xml, summary_loose_xml, on='participantId', how='left')
df_loose__result_xml.to_csv('/Users/shiyi.he/Desktop/PARSER/loose_parse_xml.csv', index=False)

## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_xml = df_result_xml.rename(columns={'valid': 'valid_strict'})
df_loose__result_xml = df_loose__result_xml.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_xml[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_xml[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_xml = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_xml.to_csv('/Users/shiyi.he/Desktop/PARSER/parser_result_xml.csv', index=False)
parser_rwsult_xml

['participantId', 'AttValue: " or \' expected, line 2, column 16 (<string>, line 2)', 'Extra content at the end of the document, line 1, column 22 (<string>, line 1)', 'Extra content at the end of the document, line 2, column 1 (<string>, line 2)', 'Extra content at the end of the document, line 3, column 12 (<string>, line 3)', 'Opening and ending tag mismatch: Patients line 1 and patient, line 2, column 16 (<string>, line 2)', 'Opening and ending tag mismatch: test line 14 and patient, line 16, column 16 (<string>, line 16)', 'Specification mandates value for attribute id1, line 16, column 14 (<string>, line 16)', "Unescaped '<' not allowed in attributes values, line 2, column 1 (<string>, line 2)", 'error parsing attribute name, line 1, column 10 (<string>, line 1)', 'error parsing attribute name, line 8, column 13 (<string>, line 8)']


,participantId,format,valid_strict,valid_loose
0,669b961e1c9cf30a26bb57d2,xml,False,True
1,60721c2da01b3a616edba9a7,xml,False,True
2,67d95269b47464c18da766ca,xml,False,True
3,6740ae39809389d7da9de209,xml,True,True
4,62fd14d7b1e0729dc8aadb79,xml,True,True
5,67e4920c40c9561040ca01d0,xml,False,True
6,677c52fe648cd68583dd3b1e,xml,True,True
7,67eff2b1a264260e948a4d14,xml,False,True
8,6501c0a2e788b5556c24569d,xml,False,True
9,66bb6b7e0ccbed81f3ed30a8,xml,False,True


In [183]:
import pandas as pd

# --- 1️⃣ 保存 strict 解析结果并展开错误统计 ---
df_result_xml = validate_xml(writing_tabular_xml)

# 按 participantId 和 errorMessage 计数，unstack→pivot
sum_xml = summarize_xml_errors(df_result_xml)
sum_xml = sum_xml.reset_index()

# 重命名第一列为 participantId（防止索引列名不一致）
sum_xml = sum_xml.rename(columns={sum_xml.columns[0]: 'participantId'})

# （可选）清除列索引名称
sum_xml.columns.name = None

print("sum_xml columns:", sum_xml.columns.tolist())

# 左右表都带 participantId，安心合并
strict_parse_xml = pd.merge(
    df_result_xml,
    sum_xml,
    on='participantId',
    how='left'
)

strict_parse_xml.to_csv(
    '/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_xml.csv',
    index=False
)


# --- 2️⃣ 保存 loose 解析结果并展开错误统计 ---
df_loose_xml = parse_loose_xml(writing_tabular_xml, code_col="code")
df_loose__result_xml = df_loose_xml[["participantId","valid","code","errorMessage"]]

# summarize_loose_errors 生成也是 pivot 形式，需要 reset_index & 重命名
summary_loose_xml = summarize_loose_errors(df_loose__result_xml)
summary_loose_xml = summary_loose_xml.reset_index()
summary_loose_xml = summary_loose_xml.rename(columns={summary_loose_xml.columns[0]: 'participantId'})
summary_loose_xml.columns.name = None

print("summary_loose_xml columns:", summary_loose_xml.columns.tolist())

loose_parse_xml = pd.merge(
    df_loose__result_xml,
    summary_loose_xml,
    on='participantId',
    how='left'
)
loose_parse_xml.to_csv(
    '/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_xml.csv',
    index=False
)


# --- 3️⃣ 合并 strict + loose 指标，输出最终结果 ---
# 重命名 valid 列，区分 strict/loose
df_result_xml = df_result_xml.rename(columns={'valid': 'valid_strict'})
df_loose__result_xml = df_loose__result_xml.rename(columns={'valid': 'valid_loose'})

# 只保留所需列
df1_subset = df_result_xml[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_xml[['participantId', 'valid_loose']]

# 合并
parser_result_xml = pd.merge(
    df1_subset,
    df2_subset,
    on='participantId',
    how='left'
)

parser_result_xml.to_csv(
    '/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_xml.csv',
    index=False
)

# 显示最终表
parser_result_xml


sum_xml columns: ['participantId', 'AttValue: " or \' expected, line 2, column 16 (<string>, line 2)', 'Extra content at the end of the document, line 1, column 22 (<string>, line 1)', 'Extra content at the end of the document, line 2, column 1 (<string>, line 2)', 'Extra content at the end of the document, line 3, column 12 (<string>, line 3)', 'Opening and ending tag mismatch: Patients line 1 and patient, line 2, column 16 (<string>, line 2)', 'Opening and ending tag mismatch: test line 14 and patient, line 16, column 16 (<string>, line 16)', 'Specification mandates value for attribute id1, line 16, column 14 (<string>, line 16)', "Unescaped '<' not allowed in attributes values, line 2, column 1 (<string>, line 2)", 'error parsing attribute name, line 1, column 10 (<string>, line 1)', 'error parsing attribute name, line 8, column 13 (<string>, line 8)']
summary_loose_xml columns: ['participantId']


,participantId,format,valid_strict,valid_loose
0,669b961e1c9cf30a26bb57d2,xml,False,True
1,60721c2da01b3a616edba9a7,xml,False,True
2,67d95269b47464c18da766ca,xml,False,True
3,6740ae39809389d7da9de209,xml,True,True
4,62fd14d7b1e0729dc8aadb79,xml,True,True
5,67e4920c40c9561040ca01d0,xml,False,True
6,677c52fe648cd68583dd3b1e,xml,True,True
7,67eff2b1a264260e948a4d14,xml,False,True
8,6501c0a2e788b5556c24569d,xml,False,True
9,66bb6b7e0ccbed81f3ed30a8,xml,False,True


## yaml

### js solution

In [163]:
import os
import json
import shutil
import subprocess
import pandas as pd
import yaml
from yaml.parser import ParserError
from yaml.scanner import ScannerError

# —— 严格解析 —— #
def validate_yaml(df, participant_col="participantId", code_col="code"):
    valids, errors = [], []
    for txt in df[code_col]:
        try:
            yaml.safe_load(txt)
            valids.append(True); errors.append(None)
        except (ParserError, ScannerError) as e:
            valids.append(False)
            errors.append(str(e).splitlines()[0])
    out = df.copy()
    out["valid"] = valids
    out["errorMessage"] = errors
    return out

def summarize_yaml_errors(df, participant_col="participantId", error_col="errorMessage"):
    records = []
    for _, r in df.iterrows():
        pid = r[participant_col]
        em  = r[error_col]
        if em:
            for line in str(em).splitlines():
                et = line.split(":",1)[0].strip()
                if et:
                    records.append({participant_col:pid, "errorType":et})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = long.groupby([participant_col,"errorType"]).size().reset_index(name="count")
    return summary.pivot(index=participant_col, columns="errorType", values="count").fillna(0).astype(int)

# —— 宽松解析 —— #
def validate_yaml_loose(
    df: pd.DataFrame,
    script_path: str = "validate_yaml_loose.js",
    participant_col: str = "participantId",
    code_col: str = "code",
    node_path: str = "/Users/shiyi.he/bin/node"
) -> pd.DataFrame:
    """
    调用 Node.js 脚本做宽松 YAML 解析。
    默认直接使用 node_path 指定的可执行文件。
    返回原 df 合并后包含：
      - valid        (bool)
      - fatal_errors (list of str)
      - warnings     (list of str)
      - parsed       (object or None)
    """
    # 1. 确认 Node.js 可执行文件
    if not os.path.isfile(node_path):
        raise RuntimeError(f"指定的 Node 可执行文件不存在：{node_path}")
    # 2. 确认脚本存在
    if not os.path.isfile(script_path):
        raise FileNotFoundError(f"找不到脚本：{script_path}")

    # 3. 准备输入
    recs = df[[participant_col, code_col]].rename(
        columns={participant_col: "participantId", code_col: "code"}
    ).to_dict(orient="records")
    inp = json.dumps(recs)

    # 4. 调用脚本
    proc = subprocess.run(
        [node_path, script_path],
        input=inp.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=os.getcwd(),
    )
    if proc.returncode != 0:
        stderr = proc.stderr.decode("utf-8", errors="ignore")
        raise RuntimeError(f"Node.js 脚本出错 (exit {proc.returncode}):\n{stderr}")

    # 5. 解析输出
    text = proc.stdout.decode("utf-8", errors="ignore").strip()
    if not text:
        raise RuntimeError("Node.js 脚本没有输出，请检查 yaml 包是否已正确安装。")
    arr = json.loads(text)
    out = pd.DataFrame(arr)

    # 6. 重命名 parsed_valid → valid
    if "parsed_valid" in out.columns:
        out = out.rename(columns={"parsed_valid": "valid"})
        out = out.rename(columns={"fatal_errors": "errorMessage"})

    # 7. 补全缺失列
    for col in ("valid", "errorMessage", "warnings", "parsed"):
        if col not in out.columns:
            out[col] = None

    # 8. 合并回原 df
    return df.merge(
        out[[participant_col, "valid", "errorMessage", "warnings", "parsed"]],
        on=participant_col,
        how="left"
    )

def summarize_loose_yaml_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将宽松解析中的 fatal_errors 拆分并 pivot 成宽表：
    行 participantId，列为每种错误类型，值为出现次数。
    """
    records = []
    for _, r in df.iterrows():
        pid = r[participant_col]
        errs = r.get(error_col) or []
        for e in errs:
            records.append({participant_col: pid, "errorType": e})
    if not records:
        return pd.DataFrame()
    long = pd.DataFrame(records)
    summary = (
        long.groupby([participant_col, "errorType"])
            .size().reset_index(name="count")
    )
    return (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0).astype(int)
    )

In [164]:
df_result_yaml=validate_yaml(writing_tabular_yaml)
sum_yaml=summarize_yaml_errors(df_result_yaml, "participantId", "errorMessage")
strict_parse_yaml = pd.merge(df_result_yaml, sum_yaml, on='participantId', how='left')
strict_parse_yaml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_yaml.csv', index=False)
strict_parse_yaml

,participantId,format,code,valid,errorMessage,"expected '<document start>', but found '<block mapping start>'",mapping values are not allowed here,sequence entries are not allowed here,while parsing a block collection,while scanning a simple key
0,67e25388d573ec2d5f1c8442,yaml,person:\r\n name: John\r\ntests\r\n # id:1\r\n normal\r\n #id: 2\r\n elavated cholesterol\r\n \r\n \r\n person:\r\n name: Michael\r\n tests\r\n #id:1\r\n normal\r\n treatment:\r\n Regular monitoring,False,while scanning a simple key,0.0,0.0,0.0,0.0,1.0
1,67dc19c31078d1581e39e75a,yaml,name: John\r\nage:\r\ntreatment: \r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\nname: Michael\r\nage:\r\ntreatment: Regular Monitoring\r\nid: 1\r\nresult: Normal,True,None,NaN,NaN,NaN,NaN,NaN
2,67adb520391eb17f214a50b4,yaml,patients:\r\n name: John\r\n tests:\r\n - id: 1\r\n - result: normal\r\n -id: 2\r\n -result: elevated cholesterol\r\n name: Michael\r\n tests:\r\n -id: 1\r\n -result: normal\r\n -treatment: regular monitoring,True,None,NaN,NaN,NaN,NaN,NaN
3,66049694dfb8828a8f3dce70,yaml,patients\r\n name: John\r\n tests: \r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevated Cholesterol\r\n name: Michael\r\n tests: \r\n id: 1\r\n result: Normal\r\n treatment: Regular Monitoring,False,mapping values are not allowed here,0.0,1.0,0.0,0.0,0.0
4,6787e4bd2bf9557a90208f03,yaml,patient: \r\n name: John\r\ntests:\r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevataed Cholesterol\r\n \r\npatients:\r\n name: Micheal\r\ntests:\r\n id: 1\r\n results: Normal\r\ntreatments: Regular Monitoring\r\n \r\n,True,None,NaN,NaN,NaN,NaN,NaN
5,67f184a3b82990251abd1592,yaml,# Example of a YAML file patients: name:Michael tests: id:\r\n - 1 result: - Normal name:John tests: id: - 1 result: - Normal id: - 2 result: - Elevated Cholesterol,False,sequence entries are not allowed here,0.0,0.0,1.0,0.0,0.0
6,60fce3df91350150e2e88423,yaml,Patients: \r\n Name: John\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Id: 2\r\n Result: Elevated Cholesterol \r\n\r\n\r\n Name: Michael\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Treatment: Regular Monitoring,False,mapping values are not allowed here,0.0,1.0,0.0,0.0,0.0
7,67d29234dbfdb6668d124115,yaml,"""petient""\r\nname: John\r\ntests:\r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\n""patient""\r\nname: Michael\r\ntests:\r\nid: 1\r\nresult: Normal\r\ntreatment: Regular Monitoring\r\n",False,"expected '<document start>', but found '<block mapping start>'",1.0,0.0,0.0,0.0,0.0
8,671a519900a99ebc15a67672,yaml,# patient\r\npatient_name: john\r\nTests:\r\n - id: 1\r\n results: Normal\r\n - id: 2\r\n results: Elevated cholesterol\r\n ptient_name: Michael\r\n tests:\r\n id:1\r\n results: normal\r\n treatment: regular Monitoring\r\n\r\n,False,while parsing a block collection,0.0,0.0,0.0,1.0,0.0
9,637d8ca00150ac6e71a34ec9,yaml,patient:\r\n John:\r\n id: 1\r\n result: normal\r\n id: 2\r\n result: elevated cholesterol\r\n Michael:\r\n id: 1\r\n result: normal\r\n treatment: regular monitoring,True,None,NaN,NaN,NaN,NaN,NaN


In [165]:
## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_yaml = validate_yaml_loose(writing_tabular_yaml, code_col="code")
df_loose__result_yaml=df_loose_yaml[["participantId","valid","parsed","errorMessage"]]
summary_loose_yaml = summarize_loose_yaml_errors(df_loose__result_yaml)
loose_parse_yaml = pd.merge(df_loose__result_yaml, summary_loose_yaml, on='participantId', how='left')
loose_parse_yaml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_yaml.csv', index=False)
loose_parse_yaml


,participantId,valid,parsed,errorMessage,A block sequence may not be used as an implicit map key,All mapping items must start at the same column,All sequence items must start at the same column,Implicit keys need to be on a single line,Implicit map keys need to be followed by map values,Nested mappings are not allowed in compact mappings,...,"Unexpected scalar token in YAML stream: ""Elevated Cholesterol""","Unexpected scalar token in YAML stream: ""John""","Unexpected scalar token in YAML stream: ""Michael""","Unexpected scalar token in YAML stream: ""Normal""","Unexpected scalar token in YAML stream: ""Regular Monitoring""","Unexpected scalar token in YAML stream: ""id""","Unexpected scalar token in YAML stream: ""name""","Unexpected scalar token in YAML stream: ""result""","Unexpected scalar token in YAML stream: ""tests""","Unexpected scalar token in YAML stream: ""treatment"""
0,67e25388d573ec2d5f1c8442,False,None,"[Implicit map keys need to be followed by map values, All mapping items must start at the same column, Implicit keys need to be on a single line, Implicit map keys need to be followed by map values, Implicit keys need to be on a single line, Implicit map keys need to be followed by map values]",0.0,1.0,0.0,2.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,67dc19c31078d1581e39e75a,True,"{'name': 'Michael', 'age': None, 'treatment': 'Regular Monitoring', 'id': 1, 'result': 'Normal'}",[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67adb520391eb17f214a50b4,True,"{'patients': {'name': 'Michael', 'tests': None, '-id': 1, '-result': 'normal', '-treatment': 'regular monitoring'}}",[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66049694dfb8828a8f3dce70,False,None,"[Implicit keys need to be on a single line, Nested mappings are not allowed in compact mappings, Implicit keys need to be on a single line, All mapping items must start at the same column, All mapping items must start at the same column, All mapping items must start at the same column]",0.0,3.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6787e4bd2bf9557a90208f03,True,"{'patient': {'name': 'John'}, 'tests': {'id': 1, 'results': 'Normal'}, 'patients': {'name': 'Micheal'}, 'treatments': 'Regular Monitoring'}",[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,67f184a3b82990251abd1592,False,None,"[Unexpected block-seq-ind on same line with key, Nested mappings are not allowed in compact mappings, Unexpected block-seq-ind on same line with key, Unexpected block-seq-ind on same line with key, Unexpected block-seq-ind on same line with key, Unexpected block-seq-ind on same line with key, Implicit map keys need to be followed by map values]",0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,60fce3df91350150e2e88423,False,None,"[Nested mappings are not allowed in compact mappings, Implicit keys need to be on a single line, Nested mappings are not allowed in compact mappings, Implicit keys need to be on a single line, Nested mappings are not allowed in compact mappings, Implicit keys need to be on a single line]",0.0,0.0,0.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,67d29234dbfdb6668d124115,False,None,"[Unexpected scalar at node end, Unexpected map-value-ind token in YAML stream: "":"", Unexpected scalar token in YAML stream: ""John"", Unexpected scalar token in YAML stream: ""tests"", Unexpected map-value-ind token in YAML stream: "":"", Unexpected scalar token in YAML stream: ""id"", Unexpected map-value-ind token in YAML stream: "":"", Unexpected scalar token in YAML stream: ""1"", Unexpected scalar token in YAML stream: ""result"", Unexpected map-value-ind token in YAML stream: "":"", Unexpected scalar token in YAML stream: ""Normal"", Unexpected scalar token in YAML stream: ""id"", Unexpected map-value-ind token in YAML stream: "":"", Unexpected scalar token in YAML stream: ""2"", Unexpected scalar token in YAML stream: ""result"",

In [166]:
## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_yaml = df_result_yaml.rename(columns={'valid': 'valid_strict'})
df_loose__result_yaml = df_loose__result_yaml.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_yaml[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_yaml[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_result_yaml = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_result_yaml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_yaml.csv', index=False)
parser_result_yaml

,participantId,format,valid_strict,valid_loose
0,67e25388d573ec2d5f1c8442,yaml,False,False
1,67dc19c31078d1581e39e75a,yaml,True,True
2,67adb520391eb17f214a50b4,yaml,True,True
3,66049694dfb8828a8f3dce70,yaml,False,False
4,6787e4bd2bf9557a90208f03,yaml,True,True
5,67f184a3b82990251abd1592,yaml,False,False
6,60fce3df91350150e2e88423,yaml,False,False
7,67d29234dbfdb6668d124115,yaml,False,False
8,671a519900a99ebc15a67672,yaml,False,False
9,637d8ca00150ac6e71a34ec9,yaml,True,True


### python solution

In [167]:
import re
import yaml
import demjson3
import pandas as pd
from yaml.parser import ParserError
from yaml.scanner import ScannerError

# ——— 严格解析 ———
def validate_yaml_py(
    df: pd.DataFrame,
    pid_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    对 df[code_col] 做最严格的 safe_load 校验。
    新增两列：
      - valid: True/False
      - errorMessage: 错误消息或 None
    """
    vs, es = [], []
    for txt in df[code_col]:
        try:
            yaml.safe_load(txt)
            vs.append(True);  es.append(None)
        except (ParserError, ScannerError) as e:
            vs.append(False)
            es.append(str(e).splitlines()[0])
    out = df.copy()
    out["valid"] = vs
    out["errorMessage"] = es
    return out

# ——— 宽松解析预处理 ———
def _preprocess_loose(txt: str) -> str:
    # 1) 删注释
    txt = re.sub(r'#.*', '', txt)
    # 2) 删行尾多余逗号
    txt = re.sub(r',\s*(?=[\]\}\n])', '', txt)
    # 3) Tab 换成两个空格
    txt = txt.replace('\t', '  ')
    return txt

# ——— 宽松解析 ———
def validate_yaml_loose_py(
    df: pd.DataFrame,
    pid_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    多阶段降级：
      A. _preprocess_loose + yaml.safe_load
      B. demjson3.decode 兜底
    新增三列：
      - valid  : True/False
      - errorMessage  : 错误摘要或 None
      - loose_parsed : 解析结果或 None
    """
    vs, es, ps = [], [], []
    for txt in df[code_col]:
        fixed = _preprocess_loose(txt)
        # A. 尝试 PyYAML
        try:
            obj = yaml.safe_load(fixed)
            vs.append(True)
            es.append(None)
            ps.append(obj)
            continue
        except Exception as ye:
            yaml_err = str(ye).splitlines()[0]
        # B. demjson3 兜底
        try:
            obj = demjson3.decode(fixed)
            vs.append(True)
            es.append(f"YAML error: {yaml_err}")
            ps.append(obj)
        except Exception as je:
            vs.append(False)
            es.append(f"YAML error: {yaml_err}; JSON fallback error: {je}")
            ps.append(None)

    out = df.copy()
    out["valid"]  = vs
    out["errorMessage"]  = es
    out["loose_parsed"] = ps
    return out

# ——— 错误汇总 ———
def summarize_errors_py(
    df: pd.DataFrame,
    pid_col: str,
    error_col: str
) -> pd.DataFrame:
    """
    将 df[error_col] 中的错误消息按 `; ` 拆解并 pivot，
    行: participantId，列: errorType，值: 出现次数
    """
    recs = []
    for _, r in df.iterrows():
        pid = r[pid_col]
        err = r[error_col]
        if err:
            for part in str(err).split("; "):
                recs.append({pid_col: pid, "errorType": part.strip()})
    if not recs:
        return pd.DataFrame()
    long = pd.DataFrame(recs)
    summary = (
        long
        .groupby([pid_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    return summary.pivot(
        index=pid_col,
        columns="errorType",
        values="count"
    ).fillna(0).astype(int)



In [168]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_yaml_py=validate_yaml_py(writing_tabular_yaml)
sum_yaml_py=summarize_errors_py(df_result_yaml_py, "participantId", "errorMessage")
strict_parse_yaml_py = pd.merge(df_result_yaml_py, sum_yaml_py, on='participantId', how='left')
strict_parse_yaml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_yaml_py.csv', index=False)
strict_parse_yaml_py


,participantId,format,code,valid,errorMessage,"expected '<document start>', but found '<block mapping start>'",mapping values are not allowed here,sequence entries are not allowed here,while parsing a block collection,while scanning a simple key
0,67e25388d573ec2d5f1c8442,yaml,person:\r\n name: John\r\ntests\r\n # id:1\r\n normal\r\n #id: 2\r\n elavated cholesterol\r\n \r\n \r\n person:\r\n name: Michael\r\n tests\r\n #id:1\r\n normal\r\n treatment:\r\n Regular monitoring,False,while scanning a simple key,0.0,0.0,0.0,0.0,1.0
1,67dc19c31078d1581e39e75a,yaml,name: John\r\nage:\r\ntreatment: \r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\nname: Michael\r\nage:\r\ntreatment: Regular Monitoring\r\nid: 1\r\nresult: Normal,True,None,NaN,NaN,NaN,NaN,NaN
2,67adb520391eb17f214a50b4,yaml,patients:\r\n name: John\r\n tests:\r\n - id: 1\r\n - result: normal\r\n -id: 2\r\n -result: elevated cholesterol\r\n name: Michael\r\n tests:\r\n -id: 1\r\n -result: normal\r\n -treatment: regular monitoring,True,None,NaN,NaN,NaN,NaN,NaN
3,66049694dfb8828a8f3dce70,yaml,patients\r\n name: John\r\n tests: \r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevated Cholesterol\r\n name: Michael\r\n tests: \r\n id: 1\r\n result: Normal\r\n treatment: Regular Monitoring,False,mapping values are not allowed here,0.0,1.0,0.0,0.0,0.0
4,6787e4bd2bf9557a90208f03,yaml,patient: \r\n name: John\r\ntests:\r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevataed Cholesterol\r\n \r\npatients:\r\n name: Micheal\r\ntests:\r\n id: 1\r\n results: Normal\r\ntreatments: Regular Monitoring\r\n \r\n,True,None,NaN,NaN,NaN,NaN,NaN
5,67f184a3b82990251abd1592,yaml,# Example of a YAML file patients: name:Michael tests: id:\r\n - 1 result: - Normal name:John tests: id: - 1 result: - Normal id: - 2 result: - Elevated Cholesterol,False,sequence entries are not allowed here,0.0,0.0,1.0,0.0,0.0
6,60fce3df91350150e2e88423,yaml,Patients: \r\n Name: John\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Id: 2\r\n Result: Elevated Cholesterol \r\n\r\n\r\n Name: Michael\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Treatment: Regular Monitoring,False,mapping values are not allowed here,0.0,1.0,0.0,0.0,0.0
7,67d29234dbfdb6668d124115,yaml,"""petient""\r\nname: John\r\ntests:\r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\n""patient""\r\nname: Michael\r\ntests:\r\nid: 1\r\nresult: Normal\r\ntreatment: Regular Monitoring\r\n",False,"expected '<document start>', but found '<block mapping start>'",1.0,0.0,0.0,0.0,0.0
8,671a519900a99ebc15a67672,yaml,# patient\r\npatient_name: john\r\nTests:\r\n - id: 1\r\n results: Normal\r\n - id: 2\r\n results: Elevated cholesterol\r\n ptient_name: Michael\r\n tests:\r\n id:1\r\n results: normal\r\n treatment: regular Monitoring\r\n\r\n,False,while parsing a block collection,0.0,0.0,0.0,1.0,0.0
9,637d8ca00150ac6e71a34ec9,yaml,patient:\r\n John:\r\n id: 1\r\n result: normal\r\n id: 2\r\n result: elevated cholesterol\r\n Michael:\r\n id: 1\r\n result: normal\r\n treatment: regular monitoring,True,None,NaN,NaN,NaN,NaN,NaN


In [169]:
## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_yaml_py = validate_yaml_loose_py(writing_tabular_yaml, code_col="code")
df_loose__result_yaml_py=df_loose_yaml_py[["participantId","valid","code","errorMessage"]]
summary_loose_yaml_py = summarize_loose_errors(df_loose__result_yaml_py)

loose_parse_yaml_py = pd.merge(df_loose__result_yaml_py, summary_loose_yaml_py, on='participantId', how='left')
loose_parse_yaml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_loose_parse_yaml_pyparse_xml.csv', index=False)
loose_parse_yaml_py


,participantId,valid,code,errorMessage,"YAML error: expected '<document start>', but found '<block mapping start>'; JSON fallback error: Unexpected text after end of JSON value","YAML error: mapping values are not allowed here; JSON fallback error: ('Unknown identifier', 'Patients')","YAML error: mapping values are not allowed here; JSON fallback error: ('Unknown identifier', 'patients')",YAML error: sequence entries are not allowed here; JSON fallback error: Spaces may not appear between a +/- number sign and the digits,"YAML error: while parsing a block collection; JSON fallback error: ('Unknown identifier', 'patient')","YAML error: while parsing a block collection; JSON fallback error: ('Unknown identifier', 'patient_name')","YAML error: while parsing a block collection; JSON fallback error: ('Unknown identifier', 'patients')","YAML error: while scanning a simple key; JSON fallback error: ('Unknown identifier', 'person')"
0,67e25388d573ec2d5f1c8442,False,person:\r\n name: John\r\ntests\r\n # id:1\r\n normal\r\n #id: 2\r\n elavated cholesterol\r\n \r\n \r\n person:\r\n name: Michael\r\n tests\r\n #id:1\r\n normal\r\n treatment:\r\n Regular monitoring,"YAML error: while scanning a simple key; JSON fallback error: ('Unknown identifier', 'person')",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,67dc19c31078d1581e39e75a,True,name: John\r\nage:\r\ntreatment: \r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\nname: Michael\r\nage:\r\ntreatment: Regular Monitoring\r\nid: 1\r\nresult: Normal,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67adb520391eb17f214a50b4,True,patients:\r\n name: John\r\n tests:\r\n - id: 1\r\n - result: normal\r\n -id: 2\r\n -result: elevated cholesterol\r\n name: Michael\r\n tests:\r\n -id: 1\r\n -result: normal\r\n -treatment: regular monitoring,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66049694dfb8828a8f3dce70,False,patients\r\n name: John\r\n tests: \r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevated Cholesterol\r\n name: Michael\r\n tests: \r\n id: 1\r\n result: Normal\r\n treatment: Regular Monitoring,"YAML error: mapping values are not allowed here; JSON fallback error: ('Unknown identifier', 'patients')",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,6787e4bd2bf9557a90208f03,True,patient: \r\n name: John\r\ntests:\r\n id: 1\r\n result: Normal\r\n id: 2\r\n result: Elevataed Cholesterol\r\n \r\npatients:\r\n name: Micheal\r\ntests:\r\n id: 1\r\n results: Normal\r\ntreatments: Regular Monitoring\r\n \r\n,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,67f184a3b82990251abd1592,False,# Example of a YAML file patients: name:Michael tests: id:\r\n - 1 result: - Normal name:John tests: id: - 1 result: - Normal id: - 2 result: - Elevated Cholesterol,YAML error: sequence entries are not allowed here; JSON fallback error: Spaces may not appear between a +/- number sign and the digits,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,60fce3df91350150e2e88423,False,Patients: \r\n Name: John\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Id: 2\r\n Result: Elevated Cholesterol \r\n\r\n\r\n Name: Michael\r\n Tests: \r\n Id: 1\r\n Result: Normal \r\n Treatment: Regular Monitoring,"YAML error: mapping values are not allowed here; JSON fallback error: ('Unknown identifier', 'Patients')",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,67d29234dbfdb6668d124115,False,"""petient""\r\nname: John\r\ntests:\r\nid: 1\r\nresult: Normal\r\nid: 2\r\nresult: Elevated Cholesterol\r\n\r\n""patient""\r\nname: Michael\r\ntests:\r\nid: 1\r\nresult: Normal\r\ntreatment: Regular Monitoring\r\n","YAML error: expected '<document start>', but found '<block mapping start>'; JSON fallback error: Unexpected text after end of JSON value",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,671a519900a99ebc15a67672,False,# patient\r\npatient_name: john\r\nTests:\r\n - id: 1\r\n results: Normal\r\n - id: 2\r\n results: Elevated cholesterol\r\n ptient_name: Michael\r\n tests:\r\n id:1\r\n results: normal\r\n treatment: regular Monitoring\r\n\r\n,"YAML error: while parsing a block collection; JSON fallback e

In [170]:
## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_yaml_py = df_result_yaml_py.rename(columns={'valid': 'valid_strict'})
df_loose__result_yaml_py = df_loose__result_yaml_py.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_yaml_py[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_yaml_py[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_yaml_py = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_yaml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_rwsult_yaml_py.csv', index=False)
parser_rwsult_yaml_py

,participantId,format,valid_strict,valid_loose
0,67e25388d573ec2d5f1c8442,yaml,False,False
1,67dc19c31078d1581e39e75a,yaml,True,True
2,67adb520391eb17f214a50b4,yaml,True,True
3,66049694dfb8828a8f3dce70,yaml,False,False
4,6787e4bd2bf9557a90208f03,yaml,True,True
5,67f184a3b82990251abd1592,yaml,False,False
6,60fce3df91350150e2e88423,yaml,False,False
7,67d29234dbfdb6668d124115,yaml,False,False
8,671a519900a99ebc15a67672,yaml,False,False
9,637d8ca00150ac6e71a34ec9,yaml,True,True


## toml

### python solution

In [171]:
import re
import toml
import demjson3
import pandas as pd

# ——— 1. 严格解析 —— #
def validate_toml_py(
    df: pd.DataFrame,
    pid_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    对 df[code_col] 做 toml.loads 严格解析：
      - strict_valid (bool)
      - strict_error (str or None)
    """
    vs, errs = [], []
    for txt in df[code_col]:
        try:
            toml.loads(txt)
            vs.append(True);  errs.append(None)
        except Exception as e:
            vs.append(False)
            errs.append(str(e).splitlines()[0])
    out = df.copy()
    out["valid"] = vs
    out["errorMessage"] = errs
    return out

# ——— 2. 宽松预处理 —— #
def _preprocess_loose_toml_py(txt: str) -> str:
    # 1) 删除尾逗号（数组、内联表末尾）
    txt = re.sub(r',\s*([\]\}])', r'\1', txt)
    # 2) 在包含空格或特殊字符的键名周围加双引号
    def _quote_key(m):
        key = m.group(1)
        return f'"{key}" ='
    txt = re.sub(r'^([^\s".=\[\]\{\},#][^=]*?)\s*=', _quote_key, txt, flags=re.MULTILINE)
    # 3) 规范等号两侧空格
    txt = re.sub(r'\s*=\s*', ' = ', txt)
    return txt

# ——— 3. 宽松解析 —— #
def validate_toml_loose_py(
    df: pd.DataFrame,
    pid_col: str = "participantId",
    code_col: str = "code"
) -> pd.DataFrame:
    """
    多阶段降级解析：
      A. _preprocess_loose_toml + toml.loads
      B. demjson3.decode 兜底 JSON5/HJSON 风格
    输出三列：
      - valid  (bool)
      - errorMessage  (str or None)
      - loose_parsed (dict/list or None)
    """
    vs, errs, parsed = [], [], []
    for txt in df[code_col]:
        fixed = _preprocess_loose_toml_py(txt)
        # 阶段 A：严格 TOML 解析
        try:
            obj = toml.loads(fixed)
            vs.append(True)
            errs.append(None)
            parsed.append(obj)
            continue
        except Exception as e:
            toml_err = str(e).splitlines()[0]
        # 阶段 B：demjson3 JSON5/HJSON 兜底
        try:
            obj = demjson3.decode(fixed)
            vs.append(True)
            errs.append(f"TOML error: {toml_err}")
            parsed.append(obj)
        except Exception as je:
            vs.append(False)
            errs.append(f"TOML error: {toml_err}; JSON fallback: {je}")
            parsed.append(None)

    out = df.copy()
    out["valid"]  = vs
    out["errorMessage"]  = errs
    out["loose_parsed"] = parsed
    return out

# ——— 4. 错误汇总 —— #
def summarize_errors_py(
    df: pd.DataFrame,
    pid_col: str,
    error_col: str
) -> pd.DataFrame:
    """
    将 df[error_col] 按 '; ' 或换行拆分并 pivot：
      行 participantId，列 errorType，值为出现次数
    """
    recs = []
    for _, r in df.iterrows():
        pid = r[pid_col]
        err = r[error_col]
        if err:
            for part in re.split(r';\s*|\r?\n', str(err)):
                tok = part.strip()
                if tok:
                    recs.append({pid_col: pid, "errorType": tok})
    if not recs:
        return pd.DataFrame()
    long = pd.DataFrame(recs)
    summary = (
        long.groupby([pid_col, "errorType"])
            .size()
            .reset_index(name="count")
    )
    return (
        summary
        .pivot(index=pid_col, columns="errorType", values="count")
        .fillna(0).astype(int)
    )

In [172]:
## 1️⃣ 1️⃣ 1️⃣ save strict result data
df_result_toml_py=validate_toml_py(writing_tabular_toml)
sum_toml_py=summarize_errors_py(df_result_toml_py, "participantId", "errorMessage")
strict_parse_toml_py = pd.merge(df_result_toml_py, sum_toml_py, on='participantId', how='left')
strict_parse_toml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_toml_py.csv', index=False)

## 2️⃣ 2️⃣ 2️⃣ save loose result data
df_loose_toml_py = validate_toml_loose_py(writing_tabular_toml, code_col="code")
df_loose__result_toml_py=df_loose_toml_py[["participantId","valid","code","errorMessage"]]
summary_loose_toml_py = summarize_loose_errors(df_loose__result_toml_py)

loose_parse_toml_py = pd.merge(df_loose__result_toml_py, summary_loose_toml_py, on='participantId', how='left')
loose_parse_toml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_loose_parse_toml_pyparse_xml.csv', index=False)


## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
df_result_toml_py = df_result_toml_py.rename(columns={'valid': 'valid_strict'})
df_loose__result_toml_py = df_loose__result_toml_py.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = df_result_toml_py[['participantId', 'format', 'valid_strict']]
df2_subset = df_loose__result_toml_py[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_toml_py = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_toml_py.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_toml_py.csv', index=False)
parser_rwsult_toml_py

,participantId,format,valid_strict,valid_loose
0,67e5cf90b79d0874e1cf16b0,toml,True,True
1,67d00f24466a67ba7e41228e,toml,False,False
2,6672d878f68b9e349b56e871,toml,False,False
3,66992f58a7964f557fc75f58,toml,False,False
4,64275e53f51a5ed4f6b7d019,toml,False,False
5,673d0377254c407e5da57f97,toml,True,True
6,5e8a5a892f6f027054a83d88,toml,True,True
7,67bf88cc763aae600b429a3e,toml,False,False
8,63e5b3e89a3823139cb9d35e,toml,False,False
9,667c2175b570aa694ce2f63c,toml,False,False


### js solution 

In [173]:
import os
import json
import shutil
import subprocess
import pandas as pd

def validate_toml_via_js(
    df: pd.DataFrame,
    script_path: str = "validate_toml.js",
    participant_col: str = "participantId",
    code_col: str = "code",
    node_path: str = "/Users/shiyi.he/bin/node"
) -> pd.DataFrame:
    """
    调用 Node.js 脚本 validate_toml.js 做严格+宽松解析。
    脚本输出字段：
      strict_valid, strict_error, strict_parsed,
      loose_valid,  loose_error,  loose_parsed
    返回原 df 合并后包含这些列。
    """
    # 1. 确认 Node.js 可执行文件
    if not os.path.isfile(node_path) or not os.access(node_path, os.X_OK):
        raise RuntimeError(f"指定的 Node 可执行文件不可用：{node_path}")
    # 2. 确认脚本存在
    if not os.path.isfile(script_path):
        raise FileNotFoundError(f"找不到脚本：{script_path}")

    # 3. 准备输入
    recs = df[[participant_col, code_col]].rename(
        columns={participant_col: "participantId", code_col: "code"}
    ).to_dict(orient="records")
    inp = json.dumps(recs)

    # 4. 调用脚本
    proc = subprocess.run(
        [node_path, script_path],
        input=inp.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=os.getcwd()
    )
    if proc.returncode != 0:
        stderr = proc.stderr.decode("utf-8", errors="ignore")
        stdout = proc.stdout.decode("utf-8", errors="ignore")
        raise RuntimeError(
            f"Node.js 脚本错误 (exit {proc.returncode}):\n"
            f"--- STDOUT ---\n{stdout}\n"
            f"--- STDERR ---\n{stderr}"
        )

    # 5. 解析输出
    text = proc.stdout.decode("utf-8", errors="ignore").strip()
    if not text:
        raise RuntimeError("Node.js 脚本未输出任何内容，请检查 yaml 包是否安装。")
    try:
        arr = json.loads(text)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"无法解析脚本输出为 JSON：{e}\n输出内容:\n{text}")

    out = pd.DataFrame(arr)

    # 6. 重命名列（如有需要）
    rename_map = {}
    if "parsed_valid" in out.columns:
        rename_map["parsed_valid"] = "loose_valid"
    if "parsed" in out.columns and "loose_parsed" not in out.columns:
        rename_map["parsed"] = "loose_parsed"
    if "parse_error" in out.columns and "loose_error" not in out.columns:
        rename_map["parse_error"] = "loose_error"
    if rename_map:
        out = out.rename(columns=rename_map)

    # 7. 补全所有预期字段
    for col in (
        "strict_valid","strict_error","strict_parsed",
        "loose_valid","loose_error","loose_parsed"
    ):
        if col not in out.columns:
            out[col] = None

    # 8. 合并回原 df
    merged = df.merge(
        out[
            [participant_col] +
            ["strict_valid","strict_error","strict_parsed"] +
            ["loose_valid","loose_error","loose_parsed"]
        ],
        on=participant_col,
        how="left"
    )
    return merged

def summarize_toml_js_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    strict_error_col: str = "strict_error",
    loose_error_col: str = "loose_error"
) -> (pd.DataFrame, pd.DataFrame):
    """
    返回两份 DataFrame：
      1. strict_summary: 严格解析的错误汇总
      2. loose_summary:  宽松解析的致命错误汇总
    行索引为 participantId，列为各 errorType，值为出现次数。
    """
    # 严格模式错误
    strict_recs = []
    for _, r in df.iterrows():
        pid = r[participant_col]
        err = r.get(strict_error_col)
        if err:
            for part in str(err).splitlines():
                et = part.split(":",1)[0].strip()
                strict_recs.append({participant_col: pid, "errorType": et})
    if strict_recs:
        df_strict_long = pd.DataFrame(strict_recs)
        strict_summary = (
            df_strict_long
            .groupby([participant_col, "errorType"])
            .size()
            .reset_index(name="count")
            .pivot(index=participant_col, columns="errorType", values="count")
            .fillna(0)
            .astype(int)
        )
    else:
        strict_summary = pd.DataFrame()

    # 宽松模式致命错误
    loose_recs = []
    for _, r in df.iterrows():
        pid = r[participant_col]
        err = r.get(loose_error_col)
        if err:
            for part in str(err).split("; "):
                et = part.strip()
                loose_recs.append({participant_col: pid, "errorType": et})
    if loose_recs:
        df_loose_long = pd.DataFrame(loose_recs)
        loose_summary = (
            df_loose_long
            .groupby([participant_col, "errorType"])
            .size()
            .reset_index(name="count")
            .pivot(index=participant_col, columns="errorType", values="count")
            .fillna(0)
            .astype(int)
        )
    else:
        loose_summary = pd.DataFrame()

    return strict_summary, loose_summary
    

df_res = validate_toml_via_js(writing_tabular_toml, code_col="code")
df_res


,participantId,format,code,strict_valid,strict_error,strict_parsed,loose_valid,loose_error,loose_parsed
0,67e5cf90b79d0874e1cf16b0,toml,"[patients]\r\n[[patients.patient]]\r\nname = ""John""\r\nage = 30\r\n\r\n[patients.patient.test]\r\nblood = ""Normal""\r\nxray = ""Normal""\r\n\r\n[[patients.patient]]\r\nname = ""Michael""\r\nage = 45\r\n\r\n[[patients.patient.test]]\r\nblood = ""Elevated Cholesterol""\r\nxray = ""Normal""\r\n\r\n[patients.patient.treatment]\r\ntype = ""Regular Monitoring""",True,None,"{'patients': {'patient': [{'name': 'John', 'age': 30, 'test': {'blood': 'Normal', 'xray': 'Normal'}}, {'name': 'Michael', 'age': 45, 'test': [{'blood': 'Elevated Cholesterol', 'xray': 'Normal'}], 'treatment': {'type': 'Regular Monitoring'}}]}}",True,None,"{'patients': {'patient': [{'name': 'John', 'age': 30, 'test': {'blood': 'Normal', 'xray': 'Normal'}}, {'name': 'Michael', 'age': 45, 'test': [{'blood': 'Elevated Cholesterol', 'xray': 'Normal'}], 'treatment': {'type': 'Regular Monitoring'}}]}}"
1,67d00f24466a67ba7e41228e,toml,"[[patients]]\r\n[[users]]\r\nname = ""john""\r\n[tests] =id ""1"" , result ""Normal""\r\nname =""Michael""\r\n[[tests]] = id ""2"", result ""Elevated cholesterol""\r\n[[[patients]]]\r\n[[users]]\r\nname = ""michael""\r\n[tests] =id ""1"" , result ""Normal\r\n[treatment] = ""regular monitoring""",False,"Unexpected character, expected only whitespace or comments till end of line at row 4, col 9, pos 49:",None,False,"TOML error: Unexpected character, expected only whitespace or comments till end of line at row 4, col 9, pos 49:; JSON5 fallback error: JSON5: invalid character 'p' at 1:3",None
2,6672d878f68b9e349b56e871,toml,name='John'test= Id=1 Results=Normal Id=2 Name='Michael' test Id=1 Results = Normal results= Elevated Cholesterol Id=2 Regula Monitoring,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 13, pos 12:",None,False,"TOML error: Unexpected character, expected only whitespace or comments till end of line at row 1, col 15, pos 14:; JSON5 fallback error: JSON5: invalid character 'a' at 1:2",None
3,66992f58a7964f557fc75f58,toml,"name= ""John""\r\n""""normal""""\r\n""""elevated cholesterol""""\r\n\r\n\r\n\r\n\r\nname= ""Michael""\r\n""""Normal""""",False,"Invalid character, expected ""="" at row 2, col 3, pos 17:",None,False,"TOML error: Invalid character, expected ""="" at row 2, col 3, pos 18:; JSON5 fallback error: JSON5: invalid character 'a' at 1:2",None
4,64275e53f51a5ed4f6b7d019,toml,"# The picture is very hard to see, so small even if I try to zoom in.\r\n# I seriously am having a hard time reading the text in the boxes.\r\n# I'll try anyway.\r\n\r\n[patient]\r\nname = ""John""\r\n[patient.tests]\r\nid = 1\r\nresult = ""Normal""\r\nid = 2\r\nresult = ""Elevated Cholesterol""\r\n\r\n[patient]\r\nname = ""Michael""\r\n[patient.tests]\r\nid = 1\r\nresult = ""Normal""\r\n[patient.treatment]\r\nisRegularMonitoring = true",False,"Can't redefine existing key at row 10, col 7, pos 238:",None,False,"TOML error: Can't redefine existing key at row 10, col 7, pos 238:; JSON5 fallback error: JSON5: invalid character '#' at 1:1",None
5,673d0377254c407e5da57f97,toml,"[[patients]]\r\nname = ""John""\r\n[[patients.tests]]\r\nid =1\r\nresult= ""Normal""\r\n[[patients.tests]]\r\nid =2\r\nresult = ""Elevated Cholesterol""\r\n[[patients]]\r\nname = ""Michael""\r\n[[patients.tests]]\r\nid =1\r\nresult= ""Normal""\r\n[[patients]]\r\ntreatment = ""Regular Monitoring""",True,None,"{'patients': [{'name': 'John', 'tests': [{'id': 1, 'result': 'Normal'}, {'id': 2, 'result': 'Elevated Cholesterol'}]}, {'name': 'Michael', 'tests': [{'id': 1, 'result': 'Normal'}]}, {'treatment': 'Regular Monitoring'}]}",True,None,"{'patients': [{'name': 'John', 'tests': [{'id': 1, 'result': 'Normal'}, {'id': 2, 'result': 'Elevated Cholesterol'}]}, {'name': 'Michael', 'tests': [{'id': 1, 'result': 'Normal'}]}, {'treatment': 'Regular Monitoring'}]}"
6,5e8a5a892f6f027054a83d88,toml,"[[patients

In [174]:
import os
import json
import shutil
import subprocess
import pandas as pd


def _run_toml_js(
    df: pd.DataFrame,
    script_path: str = "validate_toml.js",
    node_path: str = "/Users/shiyi.he/bin/node"
) -> pd.DataFrame:
    """
    Internal helper: execute the Node.js TOML script and return its full output as a DataFrame.
    """
    # Determine Node executable
    if node_path and os.path.isfile(node_path) and os.access(node_path, os.X_OK):
        node_cmd = node_path
    else:
        node_cmd = shutil.which("node") or shutil.which("nodejs")
    if not node_cmd:
        raise RuntimeError("Node executable not found; ensure Node.js is installed and in PATH.")
    
    # Ensure script exists
    if not os.path.isfile(script_path):
        raise FileNotFoundError(f"Script not found: {script_path}")
    
    # Prepare JSON input
    records = df[["participantId", "code"]].to_dict(orient="records")
    proc = subprocess.run(
        [node_cmd, script_path],
        input=json.dumps(records).encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=os.getcwd()
    )
    if proc.returncode != 0:
        stdout = proc.stdout.decode("utf-8", errors="ignore")
        stderr = proc.stderr.decode("utf-8", errors="ignore")
        raise RuntimeError(f"Node.js script error (exit {proc.returncode}):\nSTDOUT:\n{stdout}\nSTDERR:\n{stderr}")
    
    out_text = proc.stdout.decode("utf-8", errors="ignore")
    try:
        arr = json.loads(out_text)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Failed to parse JSON output: {e}\nOutput:\n{out_text}")
    
    return pd.DataFrame(arr)

def validate_toml_strict_js(
    df: pd.DataFrame,
    script_path: str = "validate_toml.js",
    node_path: str = "/Users/shiyi.he/bin/node"
) -> pd.DataFrame:
    """
    Run the Node.js TOML script and return strict parser results, renamed to uniform fields:
      - participantId
      - valid         (was strict_valid)
      - errorMessage  (was strict_error)
      - parsed        (was strict_parsed)
    """
    df_full = _run_toml_js(df, script_path, node_path)
    # Rename columns
    df_full = df_full.rename(columns={
        "strict_valid": "valid",
        "strict_error": "errorMessage",
        "strict_parsed": "parsed"
    })
    # Ensure existence
    for col in ["valid", "errorMessage", "parsed"]:
        if col not in df_full.columns:
            df_full[col] = None
    return df_full[["participantId", "valid", "errorMessage", "parsed"]]

def validate_toml_loose_js(
    df: pd.DataFrame,
    script_path: str = "validate_toml.js",
    node_path: str = "/Users/shiyi.he/bin/node"
) -> pd.DataFrame:
    """
    Run the Node.js TOML script and return loose parser results, renamed to uniform fields:
      - participantId
      - valid         (was loose_valid)
      - errorMessage  (was loose_error)
      - parsed        (was loose_parsed)
    """
    df_full = _run_toml_js(df, script_path, node_path)
    # Rename columns
    df_full = df_full.rename(columns={
        "loose_valid": "valid",
        "loose_error": "errorMessage",
        "loose_parsed": "parsed"
    })
    # Ensure existence
    for col in ["valid", "errorMessage", "parsed"]:
        if col not in df_full.columns:
            df_full[col] = None
    return df_full[["participantId", "valid", "errorMessage", "parsed"]]





def summarize_toml_errors(
    df: pd.DataFrame,
    participant_col: str = "participantId",
    error_col: str = "errorMessage"
) -> pd.DataFrame:
    """
    将 df[error_col] 中的错误消息拆分并 pivot 成宽表。
    
    - 如果是 strict_error，通常是一行或多行换行分隔；
    - 如果是 loose_error，通常用 '; ' 分隔。
    本函数统一用正则同时按换行和分号拆解。
    
    返回一个 DataFrame：
      行索引：participantId
      列：各 errorType
      值：出现次数
    """
    records = []
    for _, row in df.iterrows():
        pid = row.get(participant_col)
        err = row.get(error_col)
        if pd.notna(err) and err:
            # 同时按换行或分号拆分
            parts = re.split(r';\s*|\r?\n', str(err))
            for part in parts:
                et = part.strip()
                if et:
                    records.append({participant_col: pid, "errorType": et})
    if not records:
        return pd.DataFrame()

    long = pd.DataFrame(records)
    summary = (
        long
        .groupby([participant_col, "errorType"])
        .size()
        .reset_index(name="count")
    )
    return (
        summary
        .pivot(index=participant_col, columns="errorType", values="count")
        .fillna(0)
        .astype(int)
    )




In [175]:
    
strict_toml = validate_toml_strict_js(writing_tabular_toml)
strict_toml_summary=summarize_toml_errors(strict_toml, 'participantId','errorMessage')
strict_parse_toml = pd.merge(strict_toml, strict_toml_summary, on='participantId', how='left')
strict_parse_toml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/strict_parse_toml.csv', index=False)
strict_parse_toml


,participantId,valid,errorMessage,parsed,"Can't redefine existing key at row 10, col 7, pos 238:","Can't redefine existing key at row 11, col 16, pos 134:","Can't redefine existing key at row 4, col 51, pos 118:","Invalid boolean, expected true or false at row 1, col 33, pos 32:","Invalid character, expected ""="" at row 1, col 10, pos 9:","Invalid character, expected ""="" at row 2, col 3, pos 17:","Invalid character, expected ""="" at row 2, col 5, pos 19:","Unexpected character, expected only whitespace or comments till end of line at row 1, col 13, pos 12:","Unexpected character, expected only whitespace or comments till end of line at row 4, col 9, pos 49:","Unexpected character, expecting string, number, datetime, boolean, inline array or inline table at row 3, col 6, pos 20:"
0,67e5cf90b79d0874e1cf16b0,True,None,"{'patients': {'patient': [{'name': 'John', 'age': 30, 'test': {'blood': 'Normal', 'xray': 'Normal'}}, {'name': 'Michael', 'age': 45, 'test': [{'blood': 'Elevated Cholesterol', 'xray': 'Normal'}], 'treatment': {'type': 'Regular Monitoring'}}]}}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67d00f24466a67ba7e41228e,False,"Unexpected character, expected only whitespace or comments till end of line at row 4, col 9, pos 49:",None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,6672d878f68b9e349b56e871,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 13, pos 12:",None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,66992f58a7964f557fc75f58,False,"Invalid character, expected ""="" at row 2, col 3, pos 17:",None,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,64275e53f51a5ed4f6b7d019,False,"Can't redefine existing key at row 10, col 7, pos 238:",None,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,673d0377254c407e5da57f97,True,None,"{'patients': [{'name': 'John', 'tests': [{'id': 1, 'result': 'Normal'}, {'id': 2, 'result': 'Elevated Cholesterol'}]}, {'name': 'Michael', 'tests': [{'id': 1, 'result': 'Normal'}]}, {'treatment': 'Regular Monitoring'}]}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5e8a5a892f6f027054a83d88,True,None,"{'patients': [{'name': 'John', 'test': [{'id': 1, 'result': 'Normal'}], 'tests': [{'id': 2, 'result': 'Elevated Cholesterol'}]}, {'name': 'Michael', 'treatment': 'Regular Monitoring', 'tests': [{'id': 1, 'result': 'Normal'}]}]}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,67bf88cc763aae600b429a3e,False,"Invalid character, expected ""="" at row 1, col 10, pos 9:",None,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,63e5b3e89a3823139cb9d35e,False,"Can't redefine existing key at row 11, col 16, pos 134:",None,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,667c2175b570aa694ce2f63c,False,"Unexpected character, expecting string, number, datetime, boolean, inline array or inline table at row 3, col 6, pos 20:",None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [176]:
loose_toml  = validate_toml_loose_js(writing_tabular_toml)
loose_toml_result=loose_toml[["participantId","valid","errorMessage"]]
summary_loose_toml = summarize_toml_errors(loose_toml_result, "participantId", "errorMessage")

loose_parse_toml = pd.merge(loose_toml_result, summary_loose_toml, on='participantId', how='left')
loose_parse_toml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/loose_parse_toml.csv', index=False)

In [132]:
strict_toml

,participantId,valid_strict,errorMessage,parsed
0,659960f32faa4ff4b116de92,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 15, pos 14:",None
1,67b8636acd5ed420cc098b64,False,"Unknown character ""40"" at row 2, col 1, pos 20:",None
2,67d1cc8044430e99a7d91b4a,False,"Unterminated string at row 4, col 12, pos 47:",None
3,67d03135f855dd0ff0994265,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 13, pos 12:",None
4,5e5c993cd0394c474f7cc502,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 15, pos 14:",None
5,5b3ec4332a5fbf00014eae57,False,"Unexpected character, expected only whitespace or comments till end of line at row 9, col 8, pos 107:",None
6,6682a94b061954ae8483fb08,True,None,"{'version': '4.4.7', 'repository': {'author': {'middle_name': '', 'IsMember': True}, 'type': {'type': 'git'}}, 'engines': {'pnpm': '>=8', 'keywords': ['canvas', 'responsive', 'charts']}}"
7,67bf5ad9c5a4637861cfd21e,False,"Invalid character, expected ""="" at row 2, col 12, pos 30:",None
8,66311bae532a469f4858933e,False,"Unexpected character, expected only whitespace or comments till end of line at row 1, col 15, pos 14:",None
9,611d0afbb2a19acfd012545b,True,None,"{'Version': '4.4.7', 'Repository': {'author': {'middle_name': '', 'isMember': True}}, 'Engines': {'pnppm': '>=8', 'type': 'git'}, 'keyword': {'canvas': '', 'responsive': '', 'charts': ''}}"


In [177]:
## 3️⃣ 3️⃣ 3️⃣ save strict + loose result
strict_toml = strict_toml.rename(columns={'valid': 'valid_strict'})
strict_toml['format'] = 'toml'
loose_toml = loose_toml.rename(columns={'valid': 'valid_loose'})
# 只保留需要的列
df1_subset = strict_toml[['participantId', 'format', 'valid_strict']]
df2_subset = loose_toml[['participantId', 'valid_loose']]
# 按 participantId 进行拼接
parser_rwsult_toml = pd.merge(df1_subset, df2_subset, on='participantId', how='left')
parser_rwsult_toml.to_csv('/Users/shiyi.he/Desktop/PARSER/result_writing/parser_result_toml.csv', index=False)
parser_rwsult_toml

,participantId,format,valid_strict,valid_loose
0,67e5cf90b79d0874e1cf16b0,toml,True,True
1,67d00f24466a67ba7e41228e,toml,False,False
2,6672d878f68b9e349b56e871,toml,False,False
3,66992f58a7964f557fc75f58,toml,False,False
4,64275e53f51a5ed4f6b7d019,toml,False,False
5,673d0377254c407e5da57f97,toml,True,True
6,5e8a5a892f6f027054a83d88,toml,True,True
7,67bf88cc763aae600b429a3e,toml,False,False
8,63e5b3e89a3823139cb9d35e,toml,False,False
9,667c2175b570aa694ce2f63c,toml,False,False
